<a href="https://colab.research.google.com/github/mohyunyang/My_Kaggle/blob/main/Aerial_Cactus_Identification_Step3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> 시드값고정

In [ ]:
import torch # 파이토치 
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)                # 파이썬 난수 생성기 시드 고정
np.random.seed(seed)             # 넘파이 난수 생성기 시드 고정
torch.manual_seed(seed)          # 파이토치 난수 생성기 시드 고정 (CPU 사용 시)
torch.cuda.manual_seed(seed)     # 파이토치 난수 생성기 시드 고정 (GPU 사용 시)
torch.cuda.manual_seed_all(seed) # 파이토치 난수 생성기 시드 고정 (멀티GPU 사용 시)
torch.backends.cudnn.deterministic = True # 확정적 연산 사용
torch.backends.cudnn.benchmark = False    # 벤치마크 기능 해제
torch.backends.cudnn.enabled = False      # cudnn 사용 해제

> 장비설정

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
device

device(type='cuda')

> 데이터준비

In [ ]:
! pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"yhyunmo","key":"8fdb7bbf0193b24d0178986819fcf478"}'}

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! kaggle competitions download -c aerial-cactus-identification
! ls
! unzip aerial-cactus-identification.zip

aerial-cactus-identification.zip: Skipping, found more recently modified local copy (use --force to force download)
 aerial-cactus-identification.zip   sample_data		    train
'kaggle (1).json'		    sample_submission.csv   train.csv
'kaggle (2).json'		    test		    train.zip
 kaggle.json			    test.zip
Archive:  aerial-cactus-identification.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd

labels = pd.read_csv('train.csv')
submission = pd.read_csv('sample_submission.csv')

In [ ]:
from zipfile import ZipFile

# 훈련 이미지 데이터 압축 풀기
with ZipFile('train.zip') as zipper:
  zipper.extractall()

# 테스트 이미지 데이터 압축 풀기
with ZipFile('test.zip') as zipper:
  zipper.extractall()

> 데이터분리

In [ ]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(labels, 
                                test_size=0.1,
                                stratify=labels['has_cactus'],
                                random_state=50)

In [ ]:
print('훈련 데이터 개수 : ', len(train))
print('검증 데이터 개수 : ', len(valid))

훈련 데이터 개수 :  15750
검증 데이터 개수 :  1750


> 데이터셋 클래스정의

In [ ]:
import cv2 # OpenCV 라이브러리
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스

class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__() # 상속받은 Dataset의 생성자 호출
        # 전달받은 인수들 저장
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    # 데이터셋 크기 반환 메서드 
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드 
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]    # 이미지 ID
        img_path = self.img_dir + img_id # 이미지 파일 경로 
        image = cv2.imread(img_path)     # 이미지 파일 읽기 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        label = self.df.iloc[idx, 1]     # 이미지 레이블(타깃값)

        if self.transform is not None:
            image = self.transform(image) # 변환기가 있다면 이미지 변환
        return image, label

> 데이터셋 생성

In [ ]:
from torchvision import transforms # 이미지 변환을 위한 모듈

# 훈련 데이터용 변환기
transform_train = transforms.Compose([transforms.ToTensor(), # 1
                                      transforms.Pad(32, padding_mode='symmetric'), # 2
                                      transforms.RandomHorizontalFlip(), # 3
                                      transforms.RandomVerticalFlip(), # 4
                                      transforms.RandomRotation(10), # 5
                                      transforms.Normalize((0.485, 0.456, 0.406), # 6
                                                           (0.229, 0.224, 0.225))])

# 검증 및 테스트 데이터용 변환기
transform_test = transforms.Compose([transforms.ToTensor(),
                                    transforms.Pad(32, padding_mode = 'symmetric'),
                                    transforms.Normalize((0.485, 0.456, 0.406),  # 평균
                                                          (0.229, 0.224, 0.225))]) # 분산

transforms.Compose() 로 여러 변환기를 하나로 묶었다

- 1번) transforms.ToTensor() : 이미지를 텐서 객체로 만든다. 이어 수행되는 다른 transforms 변환기들이 텐서 객체를 입력받기 때문이다

- 2번) transforms.Pad() : 이미지 주변에 패딩을 추가한다. 여기서는 32를 전달했으니 원본이미지(32x32) 주변에 32두께의 패딩을 두른다. padding_mode ='symmetric' 은 상하좌우대칭이 되는 모양으로 만들어준다. 이를 위해 패딩을 32나 추가한 것

- 3번) transforms.RandomHorizontalFilp(), transforms.RandomVericalFlip() : 각각 무작위로 이미지를 좌우, 상하대칭 변환한다. 기본값은 0.5로 전체 이미지 중 50%를 랜덤으로 뽑아 대칭변환한다

- 4번) transforms.RandomRotation() : 이미지를 회전시킨다. 파라미터로 10을 전달하면 -10 ~ 10도 사이값 만큼 무작위로 회전한다

- 5번) transforms.Normalize() : 데이터를 지정한 평균과 분산에 맞게 정규화시킨다. 0~1사이 값으로 정해준다. 평균과 분산이 세 개씩 있는 이유는 이미지데이터 색상은 RGB로 구성되있기에 각각을 정규화해야되서 평균과 분산에 값을 세 개씩 전달한 것이다

위 평균과 분산 수치를 대체로 사용한다(실험결과성능 대체로 성능잘나옴)




> 데이터셋 및 로더 생성

In [ ]:
# 데이터셋
dataset_train = ImageDataset(df=train, img_dir='train/', transform=transform_train)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform_test)

In [ ]:
from torch.utils.data import DataLoader # 데이터 로더 클래스

loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

앞서 데이터셋을 만들 때 이미지 변환기(transforms_train and _test)를 전달했다. 그러면 데이터로더로 데이터를 불러올 때마다 이미지 변환을 수행한다.

이 때 변환기중 랜덤 변환기들은 무작위로 변환을 가하기 때문에 매번 다르게 변환한다. 

즉, 원본이미지는 같지만 첫 번째 에폭과 두 번째 에폭에서 서로다른 이미지로훈련하는 효과를 얻을 수 있다

이것이 바로 데이터 증강 기법이다

# 모델생성

데이터가 준비되었으니 CNN 모델을 설계해보자

베이스라인에는 합성곱과 최대풀링계층이 두 개씩이고 이어 평균풀링계층과 전결합계층이 하니씩 있는 구조다

이번에는 더 깊은 CNN을 만들 차례다

신경망 계층이 깊어지면 대체로 예측력이 좋아진다(지나치게 깊으면 과대적합)

아울러 배치정규화를 적용하고 활성화함수를 Leaky ReLU로 바꿔서 성능을 더 높여보겠다

> 신경망 계층 설계

In [ ]:
import torch.nn as nn # 신경망 모듈
import torch.nn.functional as F # 신경망 모듈에서 자주 사용되는 함수

class Model(nn.Module):
    # 신경망 계층 정의
    def __init__(self):
        super().__init__() # 상속받은 nn.Module의 __init__() 메서드 호출
        # 1 ~ 5번째 {합성곱, 배치 정규화, 최대 풀링} 계층 
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=32,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(32), # 배치 정규화/ 파라미터로 채널수 전달
                                    nn.LeakyReLU(), # LeakyReLU 활성화 함수
                                    nn.MaxPool2d(kernel_size=2))

        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(64),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(128),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(256),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        # 평균 풀링 계층 
        self.avg_pool = nn.AvgPool2d(kernel_size=4) 
        # 전결합 계층
        self.fc1 = nn.Linear(in_features=512 * 1 * 1, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=2)

    # 순전파 출력 정의 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.avg_pool(x)
        x = x.view(-1, 512 * 1 * 1) # 평탄화
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
model = Model().to(device)  # Model 클래스를 활용해 CNN 모델을 만든 뒤 장비에 할당

앞으로 model 은 이렇게 생각하자

(GPU)신경망계층을 정의한 클래스

# 모델훈련

베이스라인 때와 마찬가지로 손실함수와 옵티마이저를 설정한 후 훈련에 돌입

> 손실함수와 옵티마이저 설정

In [ ]:
# 손실함수
criterion = nn.CrossEntropyLoss()

옵티마이저는 adamax 를 사용 (어떤 옵티마이저가 좋은지 실험해봐야한다)

In [ ]:
# 옵티마이저
optimizer = torch.optim.Adamax(model.parameters(), lr=0.00006)

> 모델 훈련

In [ ]:
epochs = 70 # 총 에폭

# 총 에폭만큼 반복
for epoch in range(epochs):
    epoch_loss = 0 # 에폭별 손실값 초기화
    
    # '반복 횟수'만큼 반복 
    for images, labels in loader_train:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실 함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 현재 배치에서의 손실 추가
        epoch_loss += loss.item() 
        # 역전파 수행
        loss.backward()
        # 가중치 갱신
        optimizer.step()
        
    print(f'에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss/len(loader_train):.4f}')    


에폭 [1/70] - 손실값: 0.1287
에폭 [2/70] - 손실값: 0.0657
에폭 [3/70] - 손실값: 0.0480
에폭 [4/70] - 손실값: 0.0416
에폭 [5/70] - 손실값: 0.0337
에폭 [6/70] - 손실값: 0.0346
에폭 [7/70] - 손실값: 0.0314
에폭 [8/70] - 손실값: 0.0270
에폭 [9/70] - 손실값: 0.0251
에폭 [10/70] - 손실값: 0.0260
에폭 [11/70] - 손실값: 0.0228
에폭 [12/70] - 손실값: 0.0228
에폭 [13/70] - 손실값: 0.0195
에폭 [14/70] - 손실값: 0.0206
에폭 [15/70] - 손실값: 0.0200
에폭 [16/70] - 손실값: 0.0172
에폭 [17/70] - 손실값: 0.0158
에폭 [18/70] - 손실값: 0.0171
에폭 [19/70] - 손실값: 0.0164
에폭 [20/70] - 손실값: 0.0162
에폭 [21/70] - 손실값: 0.0137
에폭 [22/70] - 손실값: 0.0147
에폭 [23/70] - 손실값: 0.0125
에폭 [24/70] - 손실값: 0.0117
에폭 [25/70] - 손실값: 0.0135
에폭 [26/70] - 손실값: 0.0113
에폭 [27/70] - 손실값: 0.0123
에폭 [28/70] - 손실값: 0.0127
에폭 [29/70] - 손실값: 0.0114
에폭 [30/70] - 손실값: 0.0111
에폭 [31/70] - 손실값: 0.0096
에폭 [32/70] - 손실값: 0.0097
에폭 [33/70] - 손실값: 0.0100
에폭 [34/70] - 손실값: 0.0100
에폭 [35/70] - 손실값: 0.0091
에폭 [36/70] - 손실값: 0.0097
에폭 [37/70] - 손실값: 0.0074
에폭 [38/70] - 손실값: 0.0074
에폭 [39/70] - 손실값: 0.0069
에폭 [40/70] - 손실값: 0.0102
에폭 [41/70

> 성능검증

In [ ]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수 임포트

# 실제값과 예측 확률값을 담을 리스트 초기화
true_list = []
preds_list = []

model.eval() # 모델을 평가 상태로 설정 

with torch.no_grad(): # 기울기 계산 비활성화
    for images, labels in loader_valid:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1] # 예측 확률값
        true = labels.cpu() # 실제값 
        # 예측 확률값과 실제값을 리스트에 추가
        preds_list.extend(preds)
        true_list.extend(true)
        
# 검증 데이터 ROC AUC 점수 계산 
print(f'검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')    

검증 데이터 ROC AUC : 0.9998


# 예측 및 결과 제출

In [ ]:
dataset_test = ImageDataset(df=submission, img_dir='test/', transform=transform_test)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=False)

model.eval() 

preds = []

with torch.no_grad():
  for images, _ loader_test:
    images = images.to(device)

    outputs = model(images)

    preds_part = torch.softmax(outputs.cpu(), dim=1)[:,1].tolist()

    preds.extend(preds.part)

In [ ]:
submission['has_cactus'] = preds
submission.to_csv('cactus_sub.csv', index=False)

In [ ]:
import shutil

shutil.rmtree('./train')
shutil.rmtree('./test')